In [1]:
import time
import arviz
import numpy as np
import pandas as pd
import scipy.special as sp
import scipy.stats as sts
import stan
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Parameters
P = [3, 5, 1, 4]
N = 10 ** 4
SEED = 1729

In [3]:
# Ground truth
np.random.seed(SEED)

true_transmission_rate_mu = np.random.beta(2, 10, len(P))
true_transmission_rate_std = np.sqrt(sts.invgamma.rvs(a=100, size=len(P)))
true_transmission_rate = np.concatenate([
    sp.expit(sp.logit(mu) + np.random.normal(0, std, p))
    for p, mu, std
    in zip(P, true_transmission_rate_mu, true_transmission_rate_std)
])

true_occurrence_rate_mu = np.random.beta(2, 10, len(P))
true_occurrence_rate_std = np.sqrt(sts.invgamma.rvs(a=50, size=len(P)))
true_occurrence_rate = np.concatenate([
    sp.expit(sp.logit(mu) + np.random.normal(0, std, p))
    for p, mu, std
    in zip(P, true_occurrence_rate_mu, true_occurrence_rate_std)
])

base_rate = np.random.beta(2, 10, 1)

In [4]:
# Simulate data
data = {}
# TODO: Add covariances between same type of activity
for p in range(sum(P)):
    occurrence = np.random.binomial(1, true_occurrence_rate[p], N)
    transmission = occurrence * np.random.binomial(1, true_transmission_rate[p], N)
    data[f'O{p+1}'] = occurrence
    data[f'T{p+1}'] = transmission

data['T0'] = np.random.binomial(1, base_rate, N)
X = pd.DataFrame(data)
z = X.loc[:, X.columns.str.startswith('T')].sum(axis=1)
y = (z > 0).astype(int)
X = X.loc[:, X.columns.str.startswith('O')]

In [5]:
c = np.array([i + 1 for i, p in enumerate(P) for __ in range(p)])

In [6]:
# Stan model code
model_code = """
data {
  int<lower=0> N;                            // number of observations
  int<lower=0> M;                            // number of places
  int<lower=0> K;                            // number of classes
  int<lower=0> P[K];                         // number of places in each class
  int<lower=0, upper=1> X[N,M];              // activity occurrences
  int<lower=0, upper=1> y[N];                // transmission (tested positive)
  int<lower=1, upper=K> c[M];                // place classes
}
parameters {
  real<lower=0, upper=1> theta[M];           // transmission rates
  real<lower=0, upper=1> mu[K];              // transmission means
  real<lower=0> sigma[K];                    // transmission standard deviations
  real<lower=0, upper=1> rho;                // underlying risk
}
model {
  // Priors
  mu ~ beta(1, 5);
  sigma ~ inv_gamma(10, 1);
  rho ~ uniform(0, 1);
  // Likelihood (classes)
  for (m in 1:M) {
    theta[m] ~ normal(mu[c[m]], sigma[c[m]]);
    target += log(1 / theta[m] + 1 / (1 - theta[m])) -
      (logit(theta[m]) - logit(mu[c[m]]))^2 / (2 * sigma[c[m]]);
  }
  // Likelihood (observations)
  for (n in 1:N) {
    real s = 0.0;
    for (m in 1:M) {
      if (X[n,m] == 1) {
        s += log1m(theta[m]);
      }
    }
    s += log1m(rho);
    if (y[n] == 1) {
      target += log1m_exp(s);
    } else {
      target += s;
    }
  }
}
""" 

In [7]:
# Build model
model_data = {'N': N, 'M': sum(P), 'K': len(P), 'P': P, 
              'X': X.to_numpy(), 'y': y.to_numpy(), 'c': c}
posterior = stan.build(model_code, data=model_data, random_seed=1)

Building... This may take some time.
Messages from stanc:
  The parameter theta has no priors.
Done.


In [ ]:
# Set NUTS parameters
nuts_samples = 2000
nuts_burnin = 1000
nuts_chains = 8

# Posterior sampling
start = time.time()
fit = posterior.sample(num_samples=nuts_samples, num_warmup=nuts_burnin, num_chains=nuts_chains)
print(f"{time.time() - start:.02f} seconds elapsed")

Sampling...
     0/24000 [>---------------------------]   0%  1 sec/0     
     1/24000 [>---------------------------]   0%  1 sec/26300 
     1/24000 [>---------------------------]   0% 3 secs/52855 
     1/24000 [>---------------------------]   0% 4 secs/80448 
     1/24000 [>---------------------------]   0% 5 secs/105508
     1/24000 [>---------------------------]   0% 6 secs/132616
     1/24000 [>---------------------------]   0% 7 secs/160530
     1/24000 [>---------------------------]   0% 8 secs/185507
     1/24000 [>---------------------------]   0% 9 secs/213303
     1/24000 [>---------------------------]   0% 10 secs/239880
     1/24000 [>---------------------------]   0% 12 secs/267477
     1/24000 [>---------------------------]   0% 13 secs/296442
     1/24000 [>---------------------------]   0% 14 secs/321825
     1/24000 [>---------------------------]   0% 15 secs/350161
     1/24000 [>---------------------------]   0% 16 secs/378935
     1/24000 [>----------------------

In [ ]:
# Set model parameters
nuts_samples = 1000
nuts_burnin = 500
chains = 4

In [ ]:
# Evaluate fit
arviz.plot_trace(fit, figsize=(8, 6));

In [ ]:
fit = fit.to_frame()
print("Inference on fitted model:")
fit.describe().T

In [ ]:
# View results
print('Ground truth:')
print(true_transmission_rate)
print(true_transmission_rate_mu)
print(true_transmission_rate_std)
print(base_rate)